In [ ]:
import sys
import pandas as pd
import string
import numpy as np
import nltk

In [ ]:
import datetime

# Create the Dataframe
Create the dataset given the five files 

"data_aerzteblatt.csv"

"data_apotheke-adhoc.csv"

"data_apotheken-umschau.csv"

"data_deutsche-apotheker-zeitung.csv"

"data_pharmazeutische-zeitung.csv"

In [ ]:
#STEP 1: Read in the documents

# List with all the csv files, containing article data
document_names = ["data_aerzteblatt.csv", 
                  "data_apotheke-adhoc.csv", 
                  "data_apotheken-umschau.csv", 
                  "data_deutsche-apotheker-zeitung.csv", 
                  "data_pharmazeutische-zeitung.csv"]

# print out all the documents that ate going to be loaded
for i, document_name in enumerate(document_names,0):
    print("Document_name_{}: {}".format(i,document_name))
    
# create dataframes for all the documents
df1 = pd.read_csv(document_names[0])
df2 = pd.read_csv(document_names[1])
df3 = pd.read_csv(document_names[2])
df4 = pd.read_csv(document_names[3])
df5 = pd.read_csv(document_names[4])

# make a list of all the dataframe objects
frames = [df1, df2, df3, df4, df5]

# Create one big dataframe by conncatenating all dataframes in frames
document = pd.concat(frames,ignore_index=True)


# remove the quotationmarks from the product name
def remove_quotations(product):
    return product[1:-1]


# remove nan
document = document.dropna()
# remove articles that have no modification value
document = document.drop('isModified', axis=1)
# remove articles that are written in another language than german or not specified
document = document[document["language"] == "de"]
document = document.drop('language', axis=1)
# remove articles without a description
document = document.drop('description', axis=1)
# remove documents that don't specify the maket launch month
document = document[document["month"] != 0]
# remove all artilces without a text
document = document.dropna(subset=["text"])

document = document[~document.url.str.contains("Beipackzettel")]
document = document[~document.url.str.contains("unternehmenskommunikation")]


#  remove the quotationmarks from the product name
document["product"] = document["product"].apply(remove_quotations)
# reset the index of the dataframe
document = document.reset_index(drop=True)



# STEP 2: Convert text to lower case 
document["text_lower"] = document['text'].apply(str.lower)

# STEP 3: Remove punctuation and other symbols

# Create function to remove punctuation and other symbols
def trans(text):
    """
    Function that removes specific chartacters from a text
    
    text: the text the characters are removed from
    """
    for symbol in [string.punctuation,"–","„","“","©","(",")","-"]:
        text = text.translate(str.maketrans('', '', symbol))
    return text


# Remove the punctuation and other symbols
document["text_lower"] = document["text_lower"].apply(trans)

# create a index for the dataframe
document["index"] = range(0, len(document))

# get the column name sof the dataframe
cols = document.columns.tolist()
# rearange the columns
cols = cols[-1:] + cols[:-1]
document = document[cols]

# create document that has all the articles
document.to_csv("document.csv")

In [ ]:
# Creating the dataset for the paragraphs
dataset = pd.DataFrame(columns = ["substance","product", "year", "month", "title", "date_published", "url", "text_index","text", "text_lower","label"])

In [ ]:
def paragraphs(text, text_lower):
    """
    Generator object takes the text and the text_lower to split it into its paragraphs and yield them one after another
    """
    # use the \n symbol to slit the text into its paragraphs
    paras = text.split("\n")
    paras_lower = text_lower.split("\n")
    
    # zip the paragraphs from text and text_lower together to yield them
    for para,para_lower in zip(paras,paras_lower):
        yield para, para_lower
    
# Create an Iterator object from the document
iterator = document.iterrows()

i=0
# for each row in the document
for row in iterator:
    # loop over the paragraphs of the article
    for paragraph, paragraph_lower in paragraphs(row[1]["text"],row[1]["text_lower"]):
        # if the partagraph is bigger than 5 words, add it to the dataframe
        if len(paragraph.split()) >= 5:
            dataset.loc[i] = [row[1]["substance"],row[1]["product"],row[1]["year"],row[1]["month"],row[1]["title"],row[1]["date_published"],row[1]["url"], row[1]["index"],paragraph,paragraph_lower, 0]
            i+=1

In [ ]:
dataset.to_csv("dataset.csv")

# Incorparate the ebaid number from the file ebaid.csv

In [ ]:
# Load the ebaid file
ebaid = pd.read_csv("ebaid.csv", sep=';')

# create a dictionary object
ebaid_dict = {}
# fill the ebaid_dict with entries, the key is the substance name and the vlaue is a list with the "ebaid", "publication date " and "decision date"
for substance, eid in zip(ebaid['substance'].tolist(),zip(ebaid['ebaid'].tolist(),ebaid['dpublication'].tolist(),ebaid['ddecision'].tolist())):
    ebaid_dict[substance] = eid

# create a dictionary object
ebaid_dict_2 = {} 
# fill the ebaid_dict with entries, the key is the product name and the vlaue is a list with the "ebaid", "publication date " and "decision date"
for product, eid in zip(ebaid['brandname'].tolist(),zip(ebaid['ebaid'].tolist(),ebaid['dpublication'].tolist(),ebaid['ddecision'].tolist())):
    ebaid_dict_2[product] = eid

counter = 0

# Create three list objects for later
ebaid_list_0=[]
ebaid_list_1=[]
ebaid_list_2=[]

#create a new dataframe object
dataset_short = pd.DataFrame()

# for each substance and productname in the dataset
for i, (substance,product) in enumerate(zip(np.unique(dataset['substance'].tolist()),np.unique(dataset['product'].tolist()))):
    # check if the substance is part of the ebaid_dict
    try:
        ebaid_list = ebaid_dict[substance]
    except:
        # if it is not, check if the product anme is part of the ebaid_dict_2
        try:
            ebaid_list = ebaid_dict_2[product]
        except:
            # if it is not, use blank spaces and higher the counter ( to know how many substances could not be found in the ebaid.csv)
            ebaid_list = ["","",""]
            counter+=1
    # add the values of the dictionary key to the three lists created earlier  
    ebaid_list_0.append(ebaid_list[0])
    ebaid_list_1.append(ebaid_list[1])
    ebaid_list_2.append(ebaid_list[2])
    
# cretae a dataset that has all different substance names in it and add the ebaid, publication date and decision date to it
dataset_short['substance'] = np.unique(dataset['substance'].tolist())
dataset_short['ebaid'] =   ebaid_list_0
dataset_short['dpublication'] =   ebaid_list_1
dataset_short['ddecision'] =   ebaid_list_2

# Show how many substance ebnaids have not been found
print("Leftover:",counter)


In [ ]:
#create a transformed version of the dataset_short and make a dictionary out of it iwth the substance name as key and the rest as value
ds_T = dataset_short.set_index('substance').T.to_dict('list')

# make a list from the substance column in the dataset
l =[]
# Append the list with the values of ds_T for the specific key
for i, substance in enumerate(dataset['substance'].tolist()):
    l.append(ds_T[substance])

# add the values to the dataset
dataset["ebaid"] = [item[0] for item in l]
dataset["dpublication"] = [item[1] for item in l]
dataset["ddecision"] = [item[2] for item in l]
dataset.to_csv("dataset.csv")

## Leftover
The 10 substance that are not found in ebaid.csv are looked up manually and added to the dataset.csv.

Furthermore the dataset was manually searched for text that are no actuall news articles and thoise rows were deleted.